 Fase de ETL 

In [50]:
import pandas as pd
import numpy as np
from datetime import datetime

In [51]:
df_pacientes = pd.read_csv("../data/raw/pacientes_salud_mental.csv")
df_diagnosticos = pd.read_csv("../data/raw/diagnosticos.csv")
df_internaciones = pd.read_csv("../data/raw/internaciones.csv")

En primer lugar, se procedió a la importación de las librerías esenciales para el procesamiento y análisis de datos: pandas para la manipulación de estructuras tabulares, numpy para operaciones numéricas, y datetime para el tratamiento de variables temporales. A continuación, se efectuó la carga de las tres fuentes principales de información: la base de pacientes del sistema de salud mental, los diagnósticos registrados y las internaciones realizadas. Esta segmentación responde a una lógica de organización estructurada que permite, en etapas posteriores, la vinculación entre registros y la generación de nuevas variables derivadas, conforme a los objetivos del análisis.

In [52]:
# Ver columnas disponibles en cada dataset
print("Columnas de PACIENTES:")
print(df_pacientes.columns)

print("\nColumnas de DIAGNÓSTICOS:")
print(df_diagnosticos.columns)

print("\nColumnas de INTERNACIONES:")
print(df_internaciones.columns)

Columnas de PACIENTES:
Index(['paciente_nro', 'paciente_fecha_nacimiento', 'paciente_sexo',
       'paciente_lat', 'paciente_long', 'paciente_barrio',
       'paciente_area_referencia', 'origen_dato', 'paciente_localidad',
       'paciente_provincia'],
      dtype='object')

Columnas de DIAGNÓSTICOS:
Index(['nro_paciente', 'fecha_carga_diagnostico', 'codigo_cie_10',
       'descripcion_cie_10', 'diagnostico_manual', 'codigo_subjetivo',
       'descripcion_subjetivo', 'diagnostico_manual_subjetivo', 'subjetivo',
       'objetivo', 'problema', 'plan', 'obra_social', 'centro_atencion',
       'nombre_localidad', 'especialidad', 'servicio', 'paciente_edad_actual',
       'diagnostico_edad'],
      dtype='object')

Columnas de INTERNACIONES:
Index(['paciente_nro', 'nombre_centro', 'nombre_localidad', 'servicio',
       'fecha_ingreso', 'fecha_egreso', 'dias_total_estada',
       'especialidad_solicitante', 'fecha_ingreso_servicio',
       'fecha_egreso_al_servicio', 'dias_estada', 'especial

A fin de contextualizar el contenido informacional de cada una de las bases disponibles, se realizó una visualización preliminar de las columnas que integran los datasets de pacientes, diagnósticos e internaciones. Este procedimiento permitió detectar, en primer término, la existencia de variables redundantes, posibles claves de unión y datos relevantes para la construcción del modelo. 

In [53]:
# Realizamos el merge con la tabla intermedia de pacientes
df_merged_int = df_internaciones.merge(df_pacientes, on='paciente_nro', how='left')

# Mostramos estructura para verificar
df_merged_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40253 entries, 0 to 40252
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   paciente_nro                         40253 non-null  int64  
 1   nombre_centro                        40253 non-null  object 
 2   nombre_localidad                     40253 non-null  object 
 3   servicio                             40253 non-null  object 
 4   fecha_ingreso                        40253 non-null  object 
 5   fecha_egreso                         40253 non-null  object 
 6   dias_total_estada                    40253 non-null  int64  
 7   especialidad_solicitante             40253 non-null  object 
 8   fecha_ingreso_servicio               40253 non-null  object 
 9   fecha_egreso_al_servicio             40253 non-null  object 
 10  dias_estada                          40253 non-null  int64  
 11  especialidad_actual         

Para incorporar al análisis variables contextuales vinculadas a episodios de internación, se procedió a la unión entre la tabla de internaciones y la estructura intermedia de pacientes, utilizando como clave el identificador único paciente_nro. Esta operación permitirá incorporar información relacionada con motivos de internación, días totales de estadía, especialidades intervinientes y situación laboral, las cuales pueden funcionar como factores explicativos o moderadores del riesgo suicida.

In [54]:
# Agrupamos internaciones por paciente
df_internaciones_agg = df_internaciones.groupby('paciente_nro').agg({
    'dias_total_estada': 'mean',
    'tipo_egreso': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Desconocido',
    'especialidad_actual': lambda x: any('MENTAL' in str(i).upper() for i in x)
}).reset_index()

# Renombramos columnas para mayor claridad
df_internaciones_agg.rename(columns={
    'dias_total_estada': 'dias_estada_avg',
    'tipo_egreso': 'tipo_egreso_mas_frecuente',
    'especialidad_actual': 'internacion_salud_mental'
}, inplace=True)

# Agregamos la cantidad de internaciones
conteo = df_internaciones['paciente_nro'].value_counts().rename('n_internaciones').reset_index()
conteo.columns = ['paciente_nro', 'n_internaciones']

# Combinamos con el agregado anterior
df_internaciones_agg = df_internaciones_agg.merge(conteo, on='paciente_nro', how='left')

df_internaciones_agg.head(10)



,paciente_nro,dias_estada_avg,tipo_egreso_mas_frecuente,internacion_salud_mental,n_internaciones
0,1411,1.388060,EGRESO POR ORDEN MEDICA,False,67
1,1960,6.250000,ALTA,False,4
2,4632,2.333333,EGRESO POR ORDEN MEDICA,False,3
3,5316,2.000000,DEFUNCION,False,1
4,6209,5.500000,EGRESO POR DEFUNCION,False,4
5,6316,10.400000,ALTA,False,5
6,6517,1.000000,ALTA,False,2
7,6667,15.000000,ALTA,False,2
8,6796,5.000000,DEFUNCION,False,1
9,6823,14.000000,EGRESO POR ORDEN MEDICA,False,3


Con el propósito de enriquecer el dataset con información derivada de las trayectorias asistenciales, se elaboraron variables agregadas a partir del dataset de internaciones. Estas incluyen: la cantidad total de internaciones por paciente (n_internaciones), el promedio de días de estadía (dias_estada_avg), el tipo de egreso más frecuente (tipo_egreso_mas_frecuente) y una variable indicadora (internacion_salud_mental) que marca si el paciente fue internado alguna vez bajo una especialidad vinculada a salud mental. Estas características permiten identificar perfiles clínicos más complejos o crónicos, relevantes para la predicción del riesgo suicida.

In [55]:
import unicodedata

# Función para normalizar texto: pasa a minúsculas, remueve tildes y caracteres raros
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
        return texto
    return ""

# Lista de palabras clave asociadas a riesgo suicida
palabras_clave = ['suicidio', 'intento', 'ideacion', 'autolesion', 'no quiere vivir']

# Normalizamos el campo 'subjetivo'
df_diagnosticos['subjetivo_normalizado'] = df_diagnosticos['subjetivo'].apply(normalizar_texto)

# Creamos la variable binaria de riesgo suicida SOLO si el código CIE-10 comienza con F
def marcar_riesgo_f(row):
    texto = row['subjetivo_normalizado']
    cie = row['codigo_cie_10']
    if isinstance(texto, str) and isinstance(cie, str) and cie.startswith('F'):
        return int(any(palabra in texto for palabra in palabras_clave))
    return 0

df_diagnosticos['riesgo_suicida'] = df_diagnosticos.apply(marcar_riesgo_f, axis=1)

# Agrupamos por paciente y generamos resumen
df_diag_agg = df_diagnosticos.groupby('nro_paciente').agg({
    'riesgo_suicida': 'max',
    'codigo_cie_10': 'count'
}).rename(columns={
    'codigo_cie_10': 'n_diagnosticos'
}).reset_index()

# Vista previa
print(df_diag_agg.head())

# Totales
print("Total pacientes con riesgo suicida:", df_diag_agg['riesgo_suicida'].sum())


   nro_paciente  riesgo_suicida  n_diagnosticos
0          9599               0               2
1          9767               0               4
2         19231               0               1
3         21992               0               1
4         23219               0              15
Total pacientes con riesgo suicida: 1121


Con el propósito de incorporar al dataset información clínica relevante vinculada a los motivos de consulta, se construyeron variables agregadas a partir del dataset de diagnósticos. En particular, se diseñó una variable indicadora de riesgo suicida (`riesgo_suicida`) que se activa únicamente cuando coexisten dos condiciones: la presencia de expresiones clave en el campo subjetivo (como “suicidio”, “autolesión”, “no quiere vivir”) y la asociación con un código CIE-10 perteneciente al bloque F, correspondiente a trastornos mentales y del comportamiento.

Dado que el campo `subjetivo` se encuentra en lenguaje natural y sin control de formato, fue necesario aplicar una etapa previa de normalización (conversión a minúsculas, remoción de tildes y caracteres especiales). En contraste, el campo `codigo_cie_10` no requiere transformaciones por tratarse de un sistema estandarizado.

Además, se construyó una variable adicional (`n_diagnosticos`) que cuantifica el total de diagnósticos registrados por paciente, lo que puede reflejar continuidad en la atención o mayor complejidad clínica. Ambas variables contribuyen a robustecer la matriz de entrenamiento, aportando información sensible a la detección de poblaciones en riesgo.


In [56]:
import unicodedata

# Función de normalización del texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
        return texto
    return ""

# Aplicar normalización al campo subjetivo
df_diagnosticos["subjetivo_normalizado"] = df_diagnosticos["subjetivo"].apply(normalizar_texto)

# Crear columnas binarias por síntoma
df_diagnosticos["refiere_ansiedad"] = df_diagnosticos["subjetivo_normalizado"].str.contains("ansiedad").astype(int)
df_diagnosticos["refiere_estres"] = df_diagnosticos["subjetivo_normalizado"].str.contains("estres").astype(int)
df_diagnosticos["refiere_insomnio"] = df_diagnosticos["subjetivo_normalizado"].str.contains("insomnio").astype(int)

# Agrupar por paciente y consolidar en columnas finales
df_subjetivas_agg = df_diagnosticos.groupby("nro_paciente")[["refiere_ansiedad", "refiere_estres", "refiere_insomnio"]].max().reset_index()

# Vista previa
df_subjetivas_agg.head()

,nro_paciente,refiere_ansiedad,refiere_estres,refiere_insomnio
0,9599,0,0,0
1,9767,0,0,0
2,19231,0,0,0
3,21992,0,0,0
4,23219,0,0,0


Con el propósito de incorporar al dataset información clínica subjetiva vinculada a los motivos de consulta, se elaboraron variables agregadas a partir del campo libre subjetivo contenido en el dataset de diagnósticos. En primer lugar, se aplicó una etapa de normalización del texto —conversión a minúsculas, remoción de tildes y caracteres especiales— con el objetivo de facilitar la detección de palabras clave en lenguaje natural, habitualmente registradas sin control de formato ni estandarización.

A continuación, se identificó la presencia de los términos “ansiedad”, “estrés” e “insomnio” dentro del contenido textual normalizado, sin considerar su posición ni la cantidad de apariciones. Esta operación permitió construir, mediante agrupamiento por número de paciente (nro_paciente), tres variables indicadoras:

refiere_ansiedad: toma valor 1 si el paciente refirió ansiedad en al menos una consulta;

refiere_estres: toma valor 1 si manifestó estrés;

refiere_insomnio: toma valor 1 si mencionó insomnio.

Estas variables permiten capturar manifestaciones clínicas subjetivas que, si bien no siempre constituyen un diagnóstico formal, pueden representar signos tempranos o persistentes de sufrimiento emocional, contribuyendo a enriquecer la matriz de entrenamiento con información sensible al análisis del riesgo suicida.



In [57]:
# Ver primeros valores únicos tal como están
df_diagnosticos["obra_social"].value_counts(dropna=False).head(20)


obra_social
PROGRAMA SUMAR                                                                238443
OSEF - OSPTF -  OBRA SOCIAL DE LA PROV DE TIERRA DEL FUEGO                     72718
OSPTF - OBRA SOCIAL DE LA PROVINCIA DE TIERRA DEL FUEGO                        60732
PARTICULARES ARGENTINOS                                                        16157
SIN IDENTIFICAR                                                                16020
SIN COBERTURA                                                                  11354
OSUTHGRA- O.S. UNION TRAB. HOTELEROS Y GASTRONOMICOS DE LA REP. ARG.            8787
O.S.E.C.A.C-OBRA SOCIAL DE LOS EMPLEADOS DE COMERCIO Y ACTIVIDADES CIVILES      8639
UOM -OSUOMRA- OBRA SOCIAL DE LA UNION OBRERA METALUR. DE LA R.A.                8069
OSECAC - OBRA SOCIAL DE EMPLEADOS DE COMERCIO Y ACTIVIDADES CIVILES             7614
UNION PERSONAL- O.S. DE LA UNION DEL PERSONAL CIVIL DE LA NACION                7422
SANCOR SALUD - ASOCIACIÓN MUTUAL SANCOR SALUD        

In [58]:
import unicodedata

# Función de normalización
def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8")
    return texto

# Clasificación robusta de cobertura desde obra_social
def clasificar_cobertura(valor):
    texto = normalizar_texto(valor)

    if any(x in texto for x in ["sumar", "sin identificar", "sin cobertura", "particulares argentinos"]):
        return "SIN COBERTURA"
    elif any(x in texto for x in ["osef", "osptf", "ministerio de salud", "iosfa", "inssjp", "profe", "inst. nac. servicios sociales jubilados"]):
        return "OBRA SOCIAL ESTATAL"
    elif texto != "":
        return "OBRA SOCIAL PAGA"
    else:
        return "SIN DATOS"

# Aplicar clasificación
df_diagnosticos["cobertura_cat"] = df_diagnosticos["obra_social"].apply(clasificar_cobertura)

# Vista de distribución
print(df_diagnosticos["cobertura_cat"].value_counts(dropna=False))

df_cobertura = df_diagnosticos.groupby("nro_paciente")["cobertura_cat"].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else "SIN DATOS"
).reset_index()

cobertura_cat
SIN COBERTURA          281974
OBRA SOCIAL ESTATAL    146128
OBRA SOCIAL PAGA       101907
SIN DATOS                 257
Name: count, dtype: int64


Con el propósito de incorporar al análisis una variable que refleje el acceso a cobertura sanitaria, se procedió a clasificar las obras sociales registradas en los diagnósticos bajo tres categorías principales. Esta clasificación se realizó a partir del campo obra_social, el cual contiene denominaciones heterogéneas y no estandarizadas.

Para facilitar su análisis, se creó la variable cobertura_cat, definida de la siguiente manera:

-SIN COBERTURA: incluye expresiones tales como “programa sumar”, “sin identificar”, “sin cobertura” y “particulares argentinos”, que refieren a situaciones de atención sin intermediación de un financiador formal;

-OBRA SOCIAL ESTATAL: comprende a organismos públicos como OSEF, OSPTF, PROFE, IOSFA, INSSJP y otros sistemas de salud dependientes del Estado;

-OBRA SOCIAL PAGA: abarca al resto de las entidades, usualmente prestadoras privadas, mutuales o prepagas;

-SIN DATOS: se asigna en casos residuales donde la información no se encuentra disponible o legible.

Esta clasificación permite distinguir entre distintos niveles de acceso al sistema de salud y su posible relación con la detección, frecuencia o gravedad de los cuadros atendidos. La incorporación de esta variable busca captar desigualdades estructurales que podrían incidir en la aparición o persistencia de situaciones de riesgo.



In [59]:
# Centro de atención más frecuente por paciente
df_centro = df_diagnosticos.groupby("nro_paciente")["centro_atencion"].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()

df_centro = df_centro.rename(columns={"centro_atencion": "centro_atencion_mas_frecuente"})

print(df_centro)


       nro_paciente        centro_atencion_mas_frecuente
0              9599                                 HRRG
1              9767         HOSPITAL REGIONAL DE USHUAIA
2             19231         HOSPITAL REGIONAL DE USHUAIA
3             21992         HOSPITAL REGIONAL DE USHUAIA
4             23219         HOSPITAL REGIONAL DE USHUAIA
...             ...                                  ...
11037        325373                                 HRRG
11038        325482  CENTRO PROVINCIAL DE REHABILITACIÓN
11039        325552                                C.A.T
11040        325639                               CAPS 7
11041        325721                                C.A.T

[11042 rows x 2 columns]


Con el propósito de incorporar una variable contextual asociada a la trayectoria asistencial de cada paciente, se construyó una nueva columna que identifica el centro de atención más frecuentemente registrado en sus diagnósticos. Para ello, se aplicó una función de agregación sobre el campo centro_atencion, agrupando por nro_paciente y seleccionando la moda (valor más repetido). Esta información puede aportar indicios sobre la continuidad asistencial, el nivel de especialización del servicio o incluso la territorialidad del seguimiento clínico.

In [60]:
# Asegurar formato datetime
df_internaciones["fecha_ingreso"] = pd.to_datetime(df_internaciones["fecha_ingreso"], errors='coerce')

# Ordenar por fecha y paciente
df_orden_nivel = df_internaciones.sort_values(by=["paciente_nro", "fecha_ingreso"], ascending=[True, False])

# Obtener el último nivel de estudio registrado por paciente
df_nivel_estudio = df_orden_nivel.groupby("paciente_nro").first().reset_index()
df_nivel_estudio = df_nivel_estudio[["paciente_nro", "nivel_estudio"]]

print(df_nivel_estudio)

       paciente_nro         nivel_estudio
0              1411  PRIMARIO            
1              1960  PRIMARIO INCOMPLETO 
2              4632  SECUNDARIO          
3              5316  SIN ESPECIFICAR     
4              6209  SIN ESPECIFICAR     
...             ...                   ...
11932        325349  UNIVERSITARIO       
11933        325373  SECUNDARIO          
11934        325516  SIN ESPECIFICAR     
11935        325552  NUNCA ASISTIÓ       
11936        325732                  None

[11937 rows x 2 columns]


Con el propósito de incorporar al análisis información vinculada al contexto educativo de los pacientes, se construyó una variable categórica denominada nivel_estudio, obtenida a partir del último registro disponible en la base de internaciones. Para ello, se ordenaron cronológicamente los episodios por paciente y se extrajo el valor correspondiente a la fecha más reciente de ingreso. Esta estrategia permite capturar la situación educativa declarada más actual, la cual puede constituir un indicador relevante para el análisis del riesgo suicida, especialmente en población joven. La variable resultante incluye categorías como “PRIMARIO”, “SECUNDARIO”, “UNIVERSITARIO”, “NUNCA ASISTIÓ” y casos sin información específica.

In [61]:
# Asegurar formato datetime
df_internaciones["fecha_ingreso"] = pd.to_datetime(df_internaciones["fecha_ingreso"], errors='coerce')

# Ordenar por fecha y paciente
df_orden_nivel = df_internaciones.sort_values(by=["paciente_nro", "fecha_ingreso"], ascending=[True, False])

# Obtener el último nivel de estudio registrado por paciente
df_situacion_laboral_actual = df_orden_nivel.groupby("paciente_nro").first().reset_index()
df_situacion_laboral_actual = df_situacion_laboral_actual[["paciente_nro", "situacion_laboral_actual"]]

df_empleo = df_situacion_laboral_actual
print(df_empleo)

       paciente_nro situacion_laboral_actual
0              1411         NO BUSCA TRABAJO
1              1960              DESEMPLEADO
2              4632                     None
3              5316                     None
4              6209                     None
...             ...                      ...
11932        325349                 JUBILADO
11933        325373                     None
11934        325516                     None
11935        325552              AMA DE CASA
11936        325732                     None

[11937 rows x 2 columns]


Con el objetivo de incorporar una dimensión socioeconómica al análisis, se construyó la variable situacion_laboral_actual, obtenida del registro más reciente de internación por paciente. Esta variable refleja la condición laboral declarada, incluyendo categorías como “DESEMPLEADO”, “JUBILADO”, “NO BUSCA TRABAJO” y “AMA DE CASA”, entre otras. Su inclusión en la matriz permite observar posibles vínculos entre la situación ocupacional y la aparición de indicadores de riesgo, especialmente en contextos de vulnerabilidad social o desarraigo productivo

In [62]:
# Mostrar las primeras filas del DataFrame df_pacientes con todas sus columnas
print(df_pacientes.head())
print(df_pacientes.columns.tolist())  # Lista las columnas para ver los nombres exactos



   paciente_nro paciente_fecha_nacimiento paciente_sexo  paciente_lat  \
0          1411                1949-09-15             M    -53.787334   
1          1412                1946-01-23             F           NaN   
2          1620                1965-06-20             F           NaN   
3          1960                1937-05-22             M           NaN   
4          2498                1949-02-05             F    -53.793607   

   paciente_long paciente_barrio paciente_area_referencia origen_dato  \
0     -67.710820          Centro              CAPS 1 - RG          hc   
1            NaN             NaN                      NaN         NaN   
2            NaN             NaN                      NaN         NaN   
3            NaN             NaN                      NaN         NaN   
4     -67.711028    San Cayetano              CAPS 2 - RG          hc   

  paciente_localidad paciente_provincia  
0         RIO GRANDE   TIERRA DEL FUEGO  
1         RIO GRANDE   TIERRA DEL FUEG

In [63]:
# Mostrar las primeras filas y el tipo de dato
print(df_pacientes['paciente_fecha_nacimiento'].head())

# Ver resumen de valores nulos
print("Cantidad de valores nulos:", df_pacientes['paciente_fecha_nacimiento'].isna().sum())

# Ver valores únicos problemáticos (ejemplo: fechas anteriores a 1900)
print("Fechas anteriores a 1900:")
print(df_pacientes.loc[df_pacientes['paciente_fecha_nacimiento'] < '1900-01-01', 'paciente_fecha_nacimiento'].unique())

# Estadísticas básicas
print(df_pacientes['paciente_fecha_nacimiento'].describe())


0    1949-09-15
1    1946-01-23
2    1965-06-20
3    1937-05-22
4    1949-02-05
Name: paciente_fecha_nacimiento, dtype: object
Cantidad de valores nulos: 43
Fechas anteriores a 1900:
['1700-01-01' '1197-07-23']
count          34005
unique         18328
top       1997-01-03
freq              11
Name: paciente_fecha_nacimiento, dtype: object


In [64]:
# Aseguramos que la columna sea datetime
df_pacientes['paciente_fecha_nacimiento'] = pd.to_datetime(df_pacientes['paciente_fecha_nacimiento'], errors='coerce')

# Reemplazar fechas erróneas menores a 1900 o mayores a hoy con NaT
fecha_minima = pd.to_datetime('1900-01-01')
fecha_maxima = pd.to_datetime('today')
mask_erronea = (df_pacientes['paciente_fecha_nacimiento'] < fecha_minima) | (df_pacientes['paciente_fecha_nacimiento'] > fecha_maxima)
df_pacientes.loc[mask_erronea, 'paciente_fecha_nacimiento'] = pd.NaT

# Eliminar filas con fecha de nacimiento nula
df_pacientes = df_pacientes.dropna(subset=['paciente_fecha_nacimiento'])

# Calcular edad
fecha_referencia = fecha_maxima
df_pacientes['edad'] = (fecha_referencia - df_pacientes['paciente_fecha_nacimiento']).dt.days // 365

# Vista previa
print(df_pacientes[['paciente_nro', 'paciente_fecha_nacimiento', 'edad']].head())

   paciente_nro paciente_fecha_nacimiento  edad
0          1411                1949-09-15    75
1          1412                1946-01-23    79
2          1620                1965-06-20    60
3          1960                1937-05-22    88
4          2498                1949-02-05    76


In [65]:
#Renombrar columnas clave para unificar antes de merges
df_internaciones_agg = df_internaciones_agg.rename(columns={"nro_paciente": "paciente_nro"})
df_diag_agg = df_diag_agg.rename(columns={"nro_paciente": "paciente_nro"})
df_subjetivas_agg = df_subjetivas_agg.rename(columns={"nro_paciente": "paciente_nro"})
df_centro = df_centro.rename(columns={"nro_paciente": "paciente_nro"})
df_empleo = df_empleo.rename(columns={"nro_paciente": "paciente_nro"})
df_nivel_estudio = df_nivel_estudio.rename(columns={"nro_paciente": "paciente_nro"})
df_cobertura = df_cobertura.rename(columns={"nro_paciente": "paciente_nro"})

print(df_subjetivas_agg)
# 4. Merge de dataframes para consolidar datos
df_pacientes = df_pacientes.merge(df_internaciones_agg, on="paciente_nro", how="left") \
                           .merge(df_diag_agg, on="paciente_nro", how="left") \
                           .merge(df_nivel_estudio, on="paciente_nro", how="left") \
                           .merge(df_empleo, on="paciente_nro", how="left") \
                           .merge(df_centro, on="paciente_nro", how="left") \
                           .merge(df_cobertura, on="paciente_nro", how="left") \
														.merge(df_subjetivas_agg, on="paciente_nro", how="left")

cols_x = [col for col in df_pacientes.columns if col.endswith('_x')]
cols_y = [col for col in df_pacientes.columns if col.endswith('_y')]

# Eliminar columnas con sufijos
df_pacientes = df_pacientes.drop(columns=cols_x + cols_y)

# Verificar columnas disponibles
print("Columnas disponibles tras merge:")
print(df_pacientes.columns.tolist())

# Elegir columnas claves para el análisis
cols_finales = [
    "paciente_nro", "edad", "paciente_sexo", "paciente_localidad", "paciente_barrio",
    "cobertura_cat", "n_internaciones", "dias_estada_avg", "tipo_egreso_mas_frecuente",
    "internacion_salud_mental", "n_diagnosticos", "refiere_ansiedad", "refiere_estres",
    "refiere_insomnio", "nivel_estudio", "situacion_laboral_actual", "centro_atencion_mas_frecuente",
    "riesgo_suicida"
]

# Comprobar que todas las columnas existen para evitar error
cols_existentes = [col for col in cols_finales if col in df_pacientes.columns]
falta = set(cols_finales) - set(cols_existentes)
if falta:
    print(f"Advertencia: faltan columnas que serán omitidas: {falta}")

# Crear df_final con columnas existentes
df_final = df_pacientes[cols_existentes].copy()


# 10. Filtrar por rango de edad 12 a 29 años inclusive
df_final = df_final[df_final['edad'].between(12, 29, inclusive='both')].copy()

# 11. Vista previa
print("Vista previa df_final filtrado:")
print(df_final.head())

       paciente_nro  refiere_ansiedad  refiere_estres  refiere_insomnio
0              9599                 0               0                 0
1              9767                 0               0                 0
2             19231                 0               0                 0
3             21992                 0               0                 0
4             23219                 0               0                 0
...             ...               ...             ...               ...
11037        325373                 0               0                 0
11038        325482                 0               0                 0
11039        325552                 0               0                 0
11040        325639                 0               0                 0
11041        325721                 0               0                 0

[11042 rows x 4 columns]
Columnas disponibles tras merge:
['paciente_nro', 'paciente_fecha_nacimiento', 'paciente_sexo', 'paciente_lat'

In [66]:
df_final = df_final[df_final["edad"].between(12, 29, inclusive="both")].copy()

# Ver resumen de datos faltantes
missing_summary = pd.DataFrame({
    "missing_count": df_final.isna().sum(),
    "missing_percentage": df_final.isna().mean() * 100
}).sort_values(by="missing_percentage", ascending=False)

print("Resumen de valores faltantes en df_final:")
print(missing_summary)

# Vista previa del df final
print(df_final.head())



Resumen de valores faltantes en df_final:
                               missing_count  missing_percentage
situacion_laboral_actual                8731           85.138957
nivel_estudio                           6953           67.801073
internacion_salud_mental                6932           67.596294
n_internaciones                         6932           67.596294
dias_estada_avg                         6932           67.596294
tipo_egreso_mas_frecuente               6932           67.596294
paciente_barrio                         5555           54.168698
refiere_ansiedad                        2159           21.053145
centro_atencion_mas_frecuente           2159           21.053145
refiere_insomnio                        2159           21.053145
refiere_estres                          2159           21.053145
riesgo_suicida                          2159           21.053145
n_diagnosticos                          2159           21.053145
cobertura_cat                           2159    

Con el fin de fortalecer el análisis, integramos información proveniente de diversas fuentes relacionadas con los pacientes. En primer lugar, validamos y estandarizamos el formato de las fechas de nacimiento para asegurar una correcta gestión temporal. A partir de esta información, calculamos la edad actual de cada paciente tomando como referencia la fecha actual, y descartamos registros con datos erróneos o inconsistentes, tales como fechas anteriores a 1900 o valores nulos.

Luego, procedimos a unificar datos que abarcan variables clínicas, sociodemográficas y contextuales, provenientes de distintos conjuntos de datos, incluyendo internaciones, diagnósticos, registros subjetivos, nivel educativo, situación laboral y centro de atención más frecuente. Para garantizar la coherencia en la integración, se uniformizaron las claves de identificación mediante el renombramiento adecuado de las columnas.

Finalmente, conformamos un dataset consolidado, focalizado en la población objetivo delimitada por un rango etario específico (de 12 a 29 años inclusive), con el propósito de centrar el análisis en un grupo joven y relevante para el estudio de riesgos asociados. Esta matriz preliminar incluye variables fundamentales para el análisis, que abarcan indicadores clínicos, socioeconómicos y contextuales.

El siguiente paso consiste en realizar un análisis exploratorio de los datos faltantes, con el objetivo de fundamentar las decisiones relativas a la imputación o exclusión de valores, garantizando así la calidad y solidez del modelo subsecuente.



In [67]:
# Calcular porcentaje de valores nulos por columna
missing_percentage = df_final.isna().mean() * 100

# Calcular cantidad absoluta de valores nulos por columna
missing_count = df_final.isna().sum()

# Crear un DataFrame resumen
missing_summary = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percentage": missing_percentage
}).sort_values(by="missing_percentage", ascending=False)

# Mostrar resumen ordenado
print("Resumen de valores faltantes por columna:")
print(missing_summary)

Resumen de valores faltantes por columna:
                               missing_count  missing_percentage
situacion_laboral_actual                8731           85.138957
nivel_estudio                           6953           67.801073
internacion_salud_mental                6932           67.596294
n_internaciones                         6932           67.596294
dias_estada_avg                         6932           67.596294
tipo_egreso_mas_frecuente               6932           67.596294
paciente_barrio                         5555           54.168698
refiere_ansiedad                        2159           21.053145
centro_atencion_mas_frecuente           2159           21.053145
refiere_insomnio                        2159           21.053145
refiere_estres                          2159           21.053145
riesgo_suicida                          2159           21.053145
n_diagnosticos                          2159           21.053145
cobertura_cat                           2159    

In [68]:
## Columnas a eliminar por alta cantidad de datos faltantes
cols_a_eliminar = ["situacion_laboral_actual", "nivel_estudio", "paciente_barrio"]

# Eliminamos las columnas indicadas solo si existen (sin error)
df_final = df_final.drop(columns=cols_a_eliminar, errors='ignore')

# Variables numéricas para imputar con mediana (solo las que son efectivamente numéricas)
num_vars = [
    "n_internaciones", "dias_estada_avg", "n_diagnosticos",
    "refiere_ansiedad", "refiere_estres", "refiere_insomnio",
    "riesgo_suicida"
]

# Variables booleanas que a veces están como object (internacion_salud_mental)
if "internacion_salud_mental" in df_final.columns:
    df_final["internacion_salud_mental"] = df_final["internacion_salud_mental"].map({True: True, False: False})
    df_final["internacion_salud_mental"].fillna(False, inplace=True)

# Variables categóricas para imputar con moda
cat_vars = [
    "cobertura_cat", "paciente_localidad", "centro_atencion_mas_frecuente", "paciente_sexo",
    "tipo_egreso_mas_frecuente"
]

# Imputar numéricas
for var in num_vars:
    if var in df_final.columns:
        df_final[var] = pd.to_numeric(df_final[var], errors='coerce')
        mediana = df_final[var].median()
        df_final[var].fillna(mediana, inplace=True)

# Imputar categóricas
for var in cat_vars:
    if var in df_final.columns:
        moda = df_final[var].mode(dropna=True)
        if not moda.empty:
            df_final[var].fillna(moda[0], inplace=True)

# Verificamos si quedaron valores nulos
print("Valores nulos restantes por columna:")
print(df_final.isnull().sum())

# Vista previa tras imputación
print(df_final.head())


Valores nulos restantes por columna:
paciente_nro                     0
edad                             0
paciente_sexo                    0
paciente_localidad               0
cobertura_cat                    0
n_internaciones                  0
dias_estada_avg                  0
tipo_egreso_mas_frecuente        0
internacion_salud_mental         0
n_diagnosticos                   0
refiere_ansiedad                 0
refiere_estres                   0
refiere_insomnio                 0
centro_atencion_mas_frecuente    0
riesgo_suicida                   0
dtype: int64
      paciente_nro  edad paciente_sexo paciente_localidad  cobertura_cat  \
44            7113    12             M            USHUAIA  SIN COBERTURA   
498          21992    12             M            USHUAIA  SIN COBERTURA   
619          27291    20             M            USHUAIA  SIN COBERTURA   
631          28291    25             F            USHUAIA  SIN COBERTURA   
4698         83168    22             F       

C:\Users\Anakr\AppData\Local\Temp\ipykernel_10808\3656610719.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final["internacion_salud_mental"].fillna(False, inplace=True)
C:\Users\Anakr\AppData\Local\Temp\ipykernel_10808\3656610719.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final["internaci

In [69]:
df_final.to_csv("../data/processed/pacientes_processed.csv", index=False)
print("dataframe guardado como csv")

dataframe guardado como csv


En esta etapa, procedimos a eliminar del dataset aquellas columnas que presentaban un porcentaje elevado de valores faltantes y que, por su falta de completitud, podrían afectar la calidad del análisis. En particular, descartamos las variables “situación laboral actual”, “nivel de estudio” y “paciente barrio”, cuyas ausencias superaban el 50%, comprometiendo su representatividad y robustez.

Sin embargo, no eliminamos las variables vinculadas a internaciones, tales como “internacion_salud_mental”, “n_internaciones”, “dias_estada_avg” y “tipo_egreso_mas_frecuente”, a pesar de que exhibían un alto porcentaje de datos faltantes. Esto se debe a que la ausencia de valores en estas variables refleja la realidad clínica de la población: no todos los pacientes fueron internados, por lo cual esos datos faltantes no constituyen pérdidas sino la ausencia del evento en sí mismo. Preservar estas variables es fundamental para conservar información clínica relevante que aporte valor predictivo y explicativo en el análisis de riesgos. Por ello, optamos por mantenerlas y aplicar técnicas de imputación adecuadas para gestionar sus valores ausentes, evitando sesgar la muestra hacia pacientes internados exclusivamente.

Para el resto de las variables numéricas con datos faltantes, implementamos una imputación basada en la mediana, método robusto que mitiga el efecto de valores atípicos. En las variables categóricas, utilizamos la moda para completar las ausencias, asegurando la coherencia y consistencia del dataset. Con estas intervenciones, logramos obtener una matriz consolidada, limpia y sin valores faltantes que comprometan el análisis, sentando las bases sólidas para las etapas posteriores de exploración y modelado.